# *Deliverable #2: Create a customer travel destinations map*

## Setup & Dependencies

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json
import matplotlib as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# Check that api_key is correct
#g_key

## NOTE: Be sure to comment out this command & to delete output from file before pushing to GitHub

In [3]:
# Check gmaps installation
gmaps.__version__

'0.9.0'

## Import Source Data

In [4]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Punta Arenas,CL,-53.1500,-70.9167,33.91,33.91,86,40,4.61,scattered clouds,2022-08-05 23:29:16
1,1,Khatanga,RU,71.9667,102.5000,59.41,59.41,78,11,7.23,few clouds,2022-08-05 23:29:16
2,2,Kapaa,US,22.0752,-159.3190,83.48,82.11,70,20,11.50,few clouds,2022-08-05 23:29:17
3,3,Severo-Yeniseyskiy,RU,60.3725,93.0408,44.26,44.26,88,99,2.91,overcast clouds,2022-08-05 23:25:28
4,4,Katsuyama,JP,36.0617,136.5010,80.96,80.96,80,100,4.76,overcast clouds,2022-08-05 23:29:18


## Filter Data for Weather Preferences (Input Statements)

In [5]:
# Add new columns to DF to hold user inputs
city_data_df['min_temp'] = ""
city_data_df['max_temp'] = ""

city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date,min_temp,max_temp
0,0,Punta Arenas,CL,-53.1500,-70.9167,33.91,33.91,86,40,4.61,scattered clouds,2022-08-05 23:29:16,,
1,1,Khatanga,RU,71.9667,102.5000,59.41,59.41,78,11,7.23,few clouds,2022-08-05 23:29:16,,
2,2,Kapaa,US,22.0752,-159.3190,83.48,82.11,70,20,11.50,few clouds,2022-08-05 23:29:17,,
3,3,Severo-Yeniseyskiy,RU,60.3725,93.0408,44.26,44.26,88,99,2.91,overcast clouds,2022-08-05 23:25:28,,
4,4,Katsuyama,JP,36.0617,136.5010,80.96,80.96,80,100,4.76,overcast clouds,2022-08-05 23:29:18,,


In [6]:
# Prompt the user to enter minimum acceptable temperature 
min_temp = float(input("What is the coolest acceptable temperature for your vacation?"))

What is the coolest acceptable temperature for your vacation?65


In [7]:
# Prompt the user to enter maximum acceptable temperature 
max_temp = float(input("What is the hottest acceptable temperature for your vacation?"))

What is the hottest acceptable temperature for your vacation?95


In [10]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame.
filtered_city_data_df = city_data_df.loc[(city_data_df["Min Temp"] >= min_temp) &
                                        (city_data_df["Max Temp"] <= max_temp)]

## Clean Filtered Data

In [11]:
# Determine if there are any empty rows.
filtered_city_data_df.count()

City_ID                382
City                   382
Country                382
Lat                    382
Lng                    382
Max Temp               382
Min Temp               382
Humidity               382
Cloudiness             382
Wind Speed             382
Current Description    382
Date                   382
min_temp               382
max_temp               382
dtype: int64

In [12]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_city_data_df = filtered_city_data_df.dropna()
filtered_city_data_df.count()

City_ID                382
City                   382
Country                382
Lat                    382
Lng                    382
Max Temp               382
Min Temp               382
Humidity               382
Cloudiness             382
Wind Speed             382
Current Description    382
Date                   382
min_temp               382
max_temp               382
dtype: int64

## Retrieve hotel data (Python *Requests* & *JSON Traversals*) 

In [40]:
# Create hotel_df to store city, country, max temp, min temp, current weather conditions, & coordinates.
hotel_df = filtered_city_data_df[["City", "Country", "Max Temp", "Min Temp", "Current Description", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Min Temp,Current Description,Lat,Lng,Hotel Name
2,Kapaa,US,83.48,82.11,few clouds,22.0752,-159.3190,
4,Katsuyama,JP,80.96,80.96,overcast clouds,36.0617,136.5010,
6,Moose Factory,CA,77.00,75.18,overcast clouds,51.2585,-80.6094,
8,Aklavik,CA,77.00,77.00,broken clouds,68.2191,-135.0107,
9,Edd,ER,89.58,89.58,light rain,13.9297,41.6925,
11,Chermen,RU,65.44,65.25,mist,43.1486,44.7069,
15,Palu,ID,75.79,75.79,overcast clouds,-0.8917,119.8707,
16,Daru,PG,79.27,79.27,overcast clouds,-9.0763,143.2092,
21,Nishihara,JP,85.28,83.68,scattered clouds,26.1842,127.7558,
25,Westport,US,86.02,79.86,clear sky,41.1415,-73.3579,


In [41]:
# Set parameters (Dict Keys) to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
#    print(params)
#    print(index)
    
    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel Not found... skipping...")   

Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel No

In [42]:
# View results of JSON query
hotel_df.head(10)

,City,Country,Max Temp,Min Temp,Current Description,Lat,Lng,Hotel Name
2,Kapaa,US,83.48,82.11,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort Hotel
4,Katsuyama,JP,80.96,80.96,overcast clouds,36.0617,136.5010,Pension UPO
6,Moose Factory,CA,77.00,75.18,overcast clouds,51.2585,-80.6094,Cree Village Eco Lodge
8,Aklavik,CA,77.00,77.00,broken clouds,68.2191,-135.0107,Aklavik Inn
9,Edd,ER,89.58,89.58,light rain,13.9297,41.6925,
11,Chermen,RU,65.44,65.25,mist,43.1486,44.7069,
15,Palu,ID,75.79,75.79,overcast clouds,-0.8917,119.8707,Hotel Santika Palu
16,Daru,PG,79.27,79.27,overcast clouds,-9.0763,143.2092,Daru Lodge
21,Nishihara,JP,85.28,83.68,scattered clouds,26.1842,127.7558,Minshuku Agaihama
25,Westport,US,86.02,79.86,clear sky,41.1415,-73.3579,The Westport Inn


In [45]:
hotel_df.count()

City                   382
Country                382
Max Temp               382
Min Temp               382
Current Description    382
Lat                    382
Lng                    382
Hotel Name             382
dtype: int64

## Clean-up retrieved hotel data

In [47]:
# Determine if there are any rows where a hotel name wasn't found
cleaned_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]

In [48]:
cleaned_hotel_df.count()

City                   348
Country                348
Max Temp               348
Min Temp               348
Current Description    348
Lat                    348
Lng                    348
Hotel Name             348
dtype: int64

In [49]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
cleaned_hotel_df.to_csv(output_data_file)

## Retrieve Data for WeatherPy_vacation (marker layer) map

In [52]:
# Using the template add city name, the country code, the weather description and min & max temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Min Temp</dt><dd>{Min Temp}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in cleaned_hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = cleaned_hotel_df[["Lat", "Lng"]]

## Create the WeatherPy_vacation (marker layer) map

In [53]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Define fig object
fig = gmaps.figure()

# Add the layer to the map
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))